# RapML 

Deep learning for rap lyrics generation using Seq2Seq Learning. Inspired by machine translation.


In [ ]:
import numpy as np 
import pandas as pd
import _pickle as cPickle
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Embedding, RepeatVector, TimeDistributed, LeakyReLU, BatchNormalization
import random
from rapml_functions import *
import math

# read in the vocab
vocab = pd.read_csv('rap_vocab.csv')

# get size of vocab
vocabsize = np.max(vocab.ix[:,0].tolist())
print(vocabsize) 
del vocab

print('vocab size is' , vocabsize)
#=============================================================================
# make functions to prepare data

def zero_pad(row, hard_size = 20):
    # pass in a pandas row 
    try:
        row = row.tolist()
        size = len(row) # this is the number of dimensions of the sequence
    except:
        #print('using list type already')
        size = hard_size     

    # make a sparse array the size of vocab & sequence:
    tmp = np.zeros(size, dtype=int)

    # need to pad out the number of entries
    num_entries = len([x for x in row if str(x) != 'nan']) # how many in current sequence

    # make the non-zero row entries reversed:
    row = ([x for x in row if str(x) != 'nan'])[::-1]
    
    # pad it to length we require
    padding = size - num_entries

    for i in range(num_entries):
        # one hot this
        tmp[i+padding] = int(row[i])

    return tmp.tolist()    


def embed_and_pad(row, vocab_size = vocabsize):
    # this is for outputs...
    # pass in a pandas row 
    try:
        row = row.tolist()
    except:
        #print('using list type already')
        pass

    size = len(row) # this is the number of dimensions of the sequence

    # make a sparse array the size of vocab & sequence:
    tmp = np.zeros((size, vocab_size+1))

    # need to pad out the number of entries
    num_entries = len([x for x in row if str(x) != 'nan']) # how many in current sequence

    # pad it to length we require
    padding = size - num_entries

    for i in range(num_entries):
        # push the sequence to the left...
        # one hot this vector
        tmp[i,int(row[i])] = 1
    return tmp    


def get_sampleweight(row):
    # this is for sample weights of output...
    # pass in a pandas row 
    try:
        row = row.tolist()
    except:
        #print('using list type already')

    size = len(row) # this is the number of dimensions of the sequence

    # make an array the size of samples & sequence:
    tmp = np.zeros(size)

    # need to pad out the number of entries
    num_entries = len([x for x in row if str(x) != 'nan']) # how many in current sequence

    # pad it to length we require
    padding = size - num_entries

    for i in range(num_entries):
        # one hot this
        tmp[i] = 1
    return tmp

def get_word_index(word, LUT):
    ''' 
    takes in the word, finds it location in the LUT or assigns
    the UNK index number to it 
    '''
    try:
        # we have the word in vocab
        indx = LUT[str(word)]
    except:
        # it's not in vocabulary
        indx = LUT['UNK']

    return indx

def thresh(x,thres):
    if x > thres:
        return x
    else:
        return 0

def decode_prediction(prediction, temp = 0.1):
    # takes in one line of prediction 2D array
    # (num_seq, vocab):

    # for each sequence get maximum word activation:
    phrase = []
    conf = []

    for word in range(np.shape(prediction)[0]):
        # change prediction to have probability dist of a certain temp:
        ogpreds = prediction[word] # archive this
        prediction[word] = [thresh(x, temp) for x in prediction[word]] # rescale
        prediction[word] = [x/np.sum(prediction[word]) for x in prediction[word]]
        #idx = np.argmax(prediction[word]) # to get the max likelihood
        if True:#try:
            # choose 1 number from the sampled probabilities
            idx = np.random.choice(range(len(prediction[word])),1,p=prediction[word])[0]
            while (idx == 8378)  &  (np.count_nonzero(prediction[word]) > 1) :
                # resample from the distribution - so to make sure it's not zambia!:
                idx = np.random.choice(range(len(prediction[word])),1,p=prediction[word])[0]

            if math.isnan(prediction[word][idx]):
                continue
            else:
                # also now remove if duplicate:
                if word > 0:
                    while (str(phrase[word-1]) == str(idx))  & (np.count_nonzero(prediction[word]) > 1):
                        # resample from the distribution - so to make sure it's not a duplicate!:
                        idx = np.random.choice(range(len(prediction[word])),1,p=prediction[word])[0]  
                        
                    phrase.append(idx) # add in the confidence score and the word index
                    conf.append(ogpreds[idx]) 
                        
                else:
                    phrase.append(idx) # add in the confidence score and the word index
                    conf.append(ogpreds[idx])
        #except:
        #    print('didnt work')
    return phrase, conf

#=================================================================================

## because of RAM limitations, make the data into batches: 

def make_batch(inputs_ = pd.read_csv('inputs.csv'),targets_ = pd.read_csv('targets.csv'),batch_size = 1000):
    # they should have same number of samples

    # randomly assign 2000 index numbers to make into a batch
    idxs = random.sample(range(len(inputs_)), batch_size)

    # convert the data into their sampled form...

    ins = [] # empty list
    outs = [] 
    sampleweight = []
    for i in idxs:
        # convert the nans to none
        ins.append(zero_pad(inputs_.ix[i,:]))
        outs.append(embed_and_pad(targets_.ix[i,:]))
        sampleweight.append(get_sampleweight(targets_.ix[i,:]))

    del idxs

    return np.array(ins), np.array(outs), np.array(sampleweight) 


## BUILD MODEL


In [2]:
# make the data using the above function
ins, targs, smpweight = make_batch()

# build model architecture in keras - (Seq2Seq)
# easy and straightforward so can just use sequential model
if True == True: 
    # hyperparams
    hidden_dim1 = 256
    hidden_dim2 = 128
    embed_dim = 128
    densedim = 2048
    n_nxt = 20 # dimension of output sequences

    # Seq2Seq model structure
    model = Sequential()
    model.add(Embedding(vocabsize+1,embed_dim, input_length=n_nxt, mask_zero = True))

    # batch normalization
    model.add(BatchNormalization())
    # encoder network
    model.add(LSTM(hidden_dim1, return_sequences = True, stateful= False,forget_bias_init='one'))
    model.add(LSTM(hidden_dim2, return_sequences = True, stateful= False,forget_bias_init='one'))
    model.add(LSTM(hidden_dim2, return_sequences = False, stateful= False,forget_bias_init='one'))
    model.add(Dropout(0.2)) # no dropout
    model.add(Dense(densedim, activation='linear'))
    # batch normalization
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))
    model.add(RepeatVector(n_nxt))

    # decoder network
    model.add(LSTM(hidden_dim2,return_sequences=True,stateful=False,forget_bias_init='one'))       
    model.add(LSTM(hidden_dim2,return_sequences=True,stateful=False,forget_bias_init='one'))
    model.add(LSTM(hidden_dim2,return_sequences=True,stateful=False,forget_bias_init='one'))
    model.add(TimeDistributed(Dense(vocabsize+1, activation = "softmax")))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop',sample_weight_mode='temporal')


/home/max/anaconda3/lib/python3.5/site-packages/keras/engine/topology.py:368: UserWarning: The `regularizers` property of layers/models is deprecated. Regularization losses are now managed via the `losses` layer/model property.
  warnings.warn('The `regularizers` property of '


In [2]:
# load in model from last saved file:

from keras.models import load_model
model = load_model('rapML model9.h5')

/home/max/anaconda3/lib/python3.5/site-packages/keras/engine/topology.py:368: UserWarning: The `regularizers` property of layers/models is deprecated. Regularization losses are now managed via the `losses` layer/model property.
  warnings.warn('The `regularizers` property of '


## TRAIN MODEL

Run this cell as many times as you like

In [3]:
try:
    del ins
    del targs
except:
    print('those inputs / outputs dont exist yet')

In [ ]:
#### make the data using the above function
ins, targs, smpweight = make_batch()

# fit the model in batches:
for i in range(100000):
#for i in range(epochs):
    print('BATCH %s' % i)
    if i > 0:
        # not the first batch so make some new data
        try:
            # remove the old input data - so we have memory:
            del ins
            del targs
            del smpweight
        except:
            print('ins and targs are already deleted')
        # make some new data:
        ins, targs, smpweight = make_batch()

    # fit the model on this batch
    model.fit(ins,targs, validation_split = 0.1, batch_size=200,nb_epoch = 1, sample_weight= smpweight)
    # remove ins and targs for memory:
    del ins
    del targs
    del smpweight

## PREDICT

Interactive session for predicting following lines from what you write in.

In [24]:
# make a prediction:
print('MAKE A PREDICTION!')
# read back in vocabulary
vocab = pd.read_csv('rap_vocab.csv')
# make vocab lookup table
LUT = {  str(vocab.ix[i,1]) : vocab.ix[i,0] for i in range(len(vocab))} 
LUT['UNK'] = len(LUT) + 1 # add the UNKnown words at the end of dictionary
LUT['UNK'] = len(LUT) + 2 # add the UNKnown words at the end of dictionary

# make reverse lookup:
INVLUT = { vocab.ix[i,0] :  str(vocab.ix[i,1]) for i in range(len(vocab))} 
INVLUT[len(LUT)] = 'UNK'
INVLUT[len(LUT)+1] = 'UNK'

del vocab # for memory


close = False # 
while close == False:
    predline = input('Please enter your first line: \n')
    print('predicting next verse from: ', rap_izer(predline))
    # 
    #print('splitting into common words:')
    predline_inp = rap_izer(predline).split()

    pred_inp = [] # setup empty array
    # get each index from vocab:
    for word in predline_inp:
        pred_inp.append(get_word_index(word,LUT))

    # convert pred_inp into np array:
    pred_inp = np.array(zero_pad(pred_inp))
    #print('shape of array is: ', np.shape(pred_inp))
    # reshape maybe:
    pred_inp = np.reshape(pred_inp,(1,np.shape(pred_inp)[0]))

    #print('\nprediction input looks like:\n', pred_inp)

    # try a prediction:
    pred_result = model.predict(pred_inp)

    #print('prediction result shape is: ' , np.shape(pred_result))

    # reshape prediction:
    pred_result = np.reshape(pred_result,(np.shape(pred_result)[1],np.shape(pred_result)[2]))

    # need to decode the predictions into a sentence...
    pred, conf = decode_prediction(pred_result,temp = 0.1)
    #print(pred,conf)

    # decode index #'s into real words:
    strng = '' # store the word here
    for word_idx in pred:
        # get the word
        strng +=  str(INVLUT[int(word_idx)])
        strng += ' ' # add spaces

    print('prediction is:\n',strng)

    # continue
    choice = input('do another phrase? (y/n): ')
    if choice == 'n':
        # finish the program!
        break
        close = True


MAKE A PREDICTION!
Please enter your first line: 
It don't matter, he's dope
predicting next verse from:  it don't matter, he's dope
using list type already
prediction is:
 they don't be kill a fake yeah get high yeah it it it kickin' kickin' little kickin' little kickin' wet 
do another phrase? (y/n): n


Clearly there needs to be some more training happening. This has been left on overnight, however, it is still not capturing language too well. The first few hours training resulted in sentences generated such as  "a a a a the the a a a the ..." - so it has come a long way.

*I believe that 2  additional things need to happen in order to improve the model:*

        1) the data needs to be cleaned up, there is a lot of data that shouldn't be there as a result of my scraping algorithm, such as the targets not being the following verse / not rhyming or being on topic.
    
        2) the model should have "attention" added, which allows the decoding LSTM to look back at inputs rather than just the fixed vector created by the encoding LSTM.
        
        3) needs MORE data - a few thousand lines just isn't enough

In [7]:
model.save('NEW rapML model2.h5') 